In [ ]:
import json

file = json.load(open("train_data.json", "r"))
print(file[1])
print(len(file))

{'agent_registry': {'agents': [{'name': 'researcher', 'role': 'Collects and summarizes information from the web and internal docs.', 'cost': 'medium', 'latency': 'medium'}, {'name': 'coder', 'role': 'Writes and fixes code, runs tests, and explains changes.', 'cost': 'high', 'latency': 'high'}, {'name': 'critic', 'role': 'Reviews outputs from other agents for quality, correctness, and clarity.', 'cost': 'low', 'latency': 'low'}]}, 'state': {'task': 'Write a short technical blog post explaining what multi-agent LLM systems are.', 'status': 'in_progress', 'step': 2, 'max_steps': 8, 'context': {'user_preferences': {'tone': 'friendly', 'audience': 'intermediate developers'}}, 'history': [{'type': 'action', 'by': 'orchestrator', 'action': 'call_agent', 'target': 'researcher', 'arguments': {'query': 'Explain what multi-agent LLM systems are, list 3–4 key benefits, and mention common frameworks.'}}, {'type': 'result', 'from': 'researcher', 'content': 'A summary of multi-agent LLM systems with 

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Llama-3.2-1B-Instruct"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
import json
from datasets import Dataset

# 1) Load your JSON array file
#    train_data.json = [ { "agent_registry": ..., "state": ..., "target_action": ... }, ... ]
with open("train_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # list of dicts

SYSTEM_PROMPT = """You are ORCHESTRATOR, a supervisor coordinating a team of AI agents and tools.
You NEVER solve the task directly. You ONLY decide the next action.
Always respond with a single JSON object describing the next action.
"""

def format_example(example):
    agent_registry = example["agent_registry"]
    state = example["state"]
    target_action = example["target_action"]

    system_message = SYSTEM_PROMPT.strip()

    user_message_content = f"""Available agents (registry):
    {json.dumps(agent_registry, ensure_ascii=False, indent=2)}

    Current task state:
    {json.dumps(state, ensure_ascii=False, indent=2)}

    Decide the next action as JSON only, using this schema:
    {{
      "action": "call_agent" | "call_tool" | "ask_user" | "finish",
      "target": string or null,
      "arguments": object,
      "final_answer": string or null,
      "reason": string
    }}
    """

    assistant_message_content = json.dumps(target_action, ensure_ascii=False, separators=(",", ":"))

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message_content.strip()},
        {"role": "assistant", "content": assistant_message_content}
    ]

    # Apply the tokenizer chat template directly for training data
    # add_generation_prompt=False because it's a complete conversation turn
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False) + "<|endoftext|>"

# 2) Build the formatted text list
# Make sure tokenizer is available in global scope before this call
formatted_data = [format_example(item) for item in data]

# 3) Create the HF Dataset
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=2,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/54 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 54 | Num Epochs = 3 | Total steps = 21
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 45,088,768 of 1,280,903,168 (3.52% trained)


Step,Training Loss
2,0.337100
4,0.332600
6,0.286500
8,0.235700
10,0.202300
12,0.164200
14,0.146700
16,0.107300
18,0.082500
20,0.074900


In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "system", "content": SYSTEM_PROMPT.strip()},
    {
        "role": "user",
        "content": """Available agents (registry):
{
  "agents": [
    {
      "name": "researcher",
      "role": "Collects and summarizes information from the web and docs.",
      "cost": "medium",
      "latency": "medium"
    },
    {
      "name": "coder",
      "role": "Writes and fixes code, runs tests, and explains changes.",
      "cost": "high",
      "latency": "high"
    }
  ]
}

Current task state:
{
  "task": "Write a short explanation of what multi-agent systems are.",
  "status": "in_progress",
  "step": 1,
  "max_steps": 5,
  "context": null,
  "history": []
}

Decide the next action as JSON only, using this schema:
{
  "action": "call_agent" | "call_tool" | "ask_user" | "finish",
  "target": string or null,
  "arguments": object,
  "final_answer": string or null,
  "reason": string
}
"""
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True, # This adds the <|start_header_id|>assistant<|end_header_id|>
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print only the generated part
# Get the length of the input tokens
input_length = inputs.shape[1]

# Decode only the generated tokens (after the input_length)
generated_tokens = outputs[0, input_length:]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(response)

{"action":"call_agent","target":"researcher","arguments":{"query":"Explain what multi-agent systems are. Choose 3-5 key points."},"final_answer":null,"reason":"The researcher agent should gather and summarize foundational information before the other agents are called."}


In [ ]:
# saves adapters (or merged HF, depending on setup)
save_dir = "lora"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

('lora/tokenizer_config.json',
 'lora/special_tokens_map.json',
 'lora/chat_template.jinja',
 'lora/tokenizer.json')

In [ ]:
# from google.colab import files
# import os

# gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
# if gguf_files:
#     gguf_file = os.path.join("gguf_model", gguf_files[0])
#     print(f"Downloading: {gguf_file}")
#     files.download(gguf_file)

In [ ]:
import os
from google.colab import files

# Assuming 'save_dir' contains the path to your models folder
# If not, please define save_dir = 'path/to/your/models/folder'

if 'save_dir' not in locals():
    save_dir = 'lora' # Default to 'models' if not defined

if os.path.exists(save_dir) and os.path.isdir(save_dir):
    print(f"Downloading files from '{save_dir}':")
    downloaded_any = False
    for filename in os.listdir(save_dir):
        filepath = os.path.join(save_dir, filename)
        if os.path.isfile(filepath):
            print(f"  - {filename}")
            files.download(filepath)
            downloaded_any = True
    if not downloaded_any:
        print(f"No files found in '{save_dir}' to download.")
else:
    print(f"Directory '{save_dir}' does not exist or is not a directory.")

  - chat_template.jinja


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  - special_tokens_map.json


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  - adapter_config.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>